# Speech Emotion Recognition - Feature Extraction

Databases used

* The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS)
* Toronto emotional speech set (TESS)

### Import Libraries

Import necessary libraries

In [8]:
import glob
import os
import librosa
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

### Load all files

We will create our numpy array extracting Mel-frequency cepstral coefficients (MFCCs) while the classes to predict will be extracted from the name of the file.

#### Defining emotions to classify 

Selecting the emotions to be classified. Note that the emotions 'neutral', 'calm' and 'surprised' are only found in RAVDESS dataset and 'pleasantly surprised' or 'ps' is only available in TESS dataset. To combine all the emotions from both datasets into 8 emotion classes, we have changed 'pleasantly surprised' to 'surprised' and 'fearful' to 'fear'

In [2]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fear',
  '07':'disgust',
  '08':'surprised'
}

#defined tess emotions to test on TESS dataset only
tess_emotions=['angry','disgust','fear','ps','happy','sad']

##defined RAVDESS emotions to test on RAVDESS dataset only
ravdess_emotions=['neutral','calm','angry', 'happy','disgust','sad','fear','surprised']

observed_emotions = ['sad','angry','happy','disgust','surprised','neutral','calm','fear']

#### Feature extraction

Using librosa package we can extract the MFCC features. This function loads the file give the file path and after resampling and computing MFCC features, returns the features. We have selected the no. of MFCCs as 40.

https://librosa.org/librosa/generated/librosa.feature.mfcc.html

In [27]:
def extract_feature(file_name, mfcc=True):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    return result

#### Choosing a dataset

Choose the dataset(s) you want to load using the following function

In [21]:
def dataset_options():
    ravdess = True
    tess = True
    data = {'ravdess': ravdess, 'tess': tess}
    print(data)
    return data


#### Load data

Load data from the datasets required which is obtained by calling the function dataset__options(). Extract features from each file with the selected emotions in chosen datasets using the extract_feature() function defined.

In [23]:
def load_data(test_size=0.2): 
    x, y = [], []
    
    mfcc = True
    
    data = dataset_options()
    paths = []
    
    if data['ravdess']:
        paths.append(r"C:\Users\ACER\Downloads\CNDMCK\Audio\Ravdess\Actor_*\*.wav")
        
    for path in paths:
        for file in glob.glob(path):
            file_name = os.path.basename(file)
            emotion = emotions[file_name.split("-")[2]]
            if emotion not in observed_emotions:
                continue
            feature = extract_feature(file, mfcc)
            if feature.size == 0:
                continue
            x.append(feature)
            y.append(emotion)
    
    if data['tess']:
        for file in glob.glob(r"C:\Users\ACER\Downloads\CNDMCK\Audio\TESS Toronto emotional speech set data\*AF_*\*.wav"):
            file_name = os.path.basename(file)
            emotion = file_name.split("_")[2][:-4] 
            if emotion == 'ps':
                emotion = 'surprised'
            if emotion not in observed_emotions:
                continue
            feature = extract_feature(file, mfcc)
            if feature.size == 0:
                continue
            x.append(feature)
            y.append(emotion)
    
    x = np.array(x)
    y = np.array(y)
    
    if test_size > 0 and test_size < 1:
        return train_test_split(x, y, test_size=test_size, random_state=42)
    
    return {"X": x, "y": y}


In [31]:
start_time = time.time()

X_train, X_test, y_train, y_test = load_data(test_size=0.3)

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

{'ravdess': True, 'tess': True}
--- Data loaded. Loading time: 105.68021988868713 seconds ---


In [35]:
X = pd.DataFrame(X_train)
y = pd.DataFrame(y_train)

In [36]:
X.shape, y.shape

((4628, 40), (4628, 1))

In [37]:
#renaming the label column to emotion
y=y.rename(columns= {0: 'emotion'})

In [38]:
#concatinating the attributes and label into a single dataframe
data = pd.concat([X, y], axis =1)

In [39]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
0,-431.243591,61.921501,9.751375,1.418937,-12.196798,3.748008,-17.486868,5.173080,-22.647293,1.887928,...,-0.156624,-1.322961,-5.576677,1.478981,2.641849,-4.475389,-0.878444,-3.821297,0.388660,fear
1,-423.856323,74.048065,22.511642,43.370129,-4.376185,14.910736,-7.447438,-0.875301,0.658448,-1.253023,...,11.257979,8.610000,8.186172,3.266101,1.137581,1.695827,0.648322,4.733205,2.931621,sad
2,-440.094177,102.067924,10.340258,1.027507,-1.292437,-0.338838,-17.597439,7.701941,-23.524864,0.057053,...,-0.619042,-0.432404,0.116134,6.217259,4.475646,8.476109,8.896357,9.631824,5.617397,surprised
3,-402.145447,72.167290,-10.806440,22.636204,-7.473465,-6.302497,-16.086420,-4.700330,-13.415092,0.882514,...,1.472952,3.017743,-0.043461,4.730020,2.318131,4.943650,4.500614,4.626682,5.656787,disgust
4,-318.353851,34.182625,-15.095026,34.088593,-8.727753,-7.272490,-13.546126,-4.402505,-3.859099,-4.075391,...,5.230363,-2.069111,-5.220823,-2.010141,2.591809,1.111715,-1.476474,-2.530045,1.001491,happy


## Shuffling data

In [40]:
#reindexing to shuffle the data at random
data = data.reindex(np.random.permutation(data.index))

In [41]:
# Storing shuffled ravdess and tess data to avoid loading again
data.to_csv("RAVTESS_MFCC_Observed.csv")